# Lac Pavin phylo

In [1]:
import matplotlib, re, os, glob, datetime, difflib, random, time, math, json, wget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import subprocess as sp
from collections import defaultdict
sns.set('notebook')
%matplotlib inline 
# hide warnings
import warnings
warnings.filterwarnings('ignore')
from Bio import SeqIO, SeqUtils, SearchIO

In [2]:
def cmdir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def scaffold(gene):
    if gene != "None":
        try: return re.search("(.+?)_[0-9]+$", gene).group(1)
        except: print(gene)

def encode(path):
    return "A" + "".join(os.path.basename(path).split("_")[1:3])      

In [3]:
rootdir = "/groups/banfield/projects/environmental/LacPavin/analysis/"

# assign taxonomy

### re-run gtdb-tk

In [10]:
cmdir(rootdir + "genomes/secondary_gtdbtk")

In [11]:
genomes = glob.glob(rootdir + "genomes/final_genomes/*")
n = math.ceil(len(genomes)/5)

for a, i in enumerate(range(0, len(genomes), n)):
    with open(rootdir + "genomes/secondary_gtdbtk/batch%d.txt" %(a), "w") as out:
        for genome in genomes[i:i + n]:
            out.write(genome + "\t" + os.path.basename(genome).split(".fa")[0] + "\n")

In [13]:
for batchfile in glob.glob(rootdir + "genomes/secondary_gtdbtk/batch*.txt"):
    dirname = rootdir + "genomes/secondary_gtdbtk/%s" %(os.path.basename(batchfile).replace(".txt", ""))
    call = "sbatch --wrap '/shared/software/bin/gtdbtk classify_wf --cpus 48 -x .fa --batchfile %s --out_dir %s'" %(batchfile, dirname)
    #print(call)

In [699]:
# redo for alti
with open(rootdir + "genomes/secondary_gtdbtk/batch5.txt", "w") as out:
    out.write("/groups/banfield/projects/environmental/LacPavin/analysis/genomes/final_genomes/A0920SED5_metabat_386.refined.fa\tA0920SED5_metabat_386.refined")
    
call = "gtdbtk classify_wf --cpus 20 -x .fa --batchfile %s --out_dir %s" %(rootdir + "genomes/secondary_gtdbtk/batch5.txt", rootdir + "genomes/secondary_gtdbtk/batch5/")
print(call)

gtdbtk classify_wf --cpus 20 -x .fa --batchfile /groups/banfield/projects/environmental/LacPavin/analysis/genomes/secondary_gtdbtk/batch5.txt --out_dir /groups/banfield/projects/environmental/LacPavin/analysis/genomes/secondary_gtdbtk/batch5/


In [4]:
# parse
gtresults = pd.concat([pd.read_csv(item, sep="\t") for item in glob.glob(rootdir + "genomes/secondary_gtdbtk/batch*/gtdbtk.*.summary.tsv")])
# remove bad alti
gtresults = gtresults[gtresults["user_genome"] != "A0920SED5_metabat_204.refined"]
gtresults["genome_slug"] = gtresults["user_genome"].apply(lambda x: x.replace(".refined", "").replace("_contigs", ""))
gtresults["phylum"] = gtresults["classification"].apply(lambda x: x.split(";")[1])

In [5]:
len(gtresults)

738

### extract cpr/dpann

In [49]:
cmdir(rootdir + "phylo")

In [704]:
for group in ["cpr", "dpann"]:
    
    basedir = rootdir + "phylo/" + group + "/"
    cmdir(basedir)
    
    if group == "cpr":
        phyla = ["p__Patescibacteria"]
    else:
        phyla = ["p__Nanoarchaeota", "p__Huberarchaeota", "p__UAP2", "p__EX4484-52",
                "p__Nanohaloarchaeota", "p__Iainarchaeota", 
                "p__Aenigmatarchaeota", "p__Micrarchaeota"]
    
    # copy over proteomes
    cmdir(basedir + "/proteins")
    all_proteins = open(basedir + "all_" + group + "_proteins.faa", "w")
    
    for key, row in gtresults[(gtresults["phylum"].isin(phyla))].iterrows():
        
        ppath = glob.glob(rootdir + "genomes/secondary_gtdbtk/batch*/identify/" + \
            "intermediate_results/marker_genes/" + row["user_genome"] + "/*_protein.faa")
        
        with open(basedir + "proteins/" + row["user_genome"].split(".")[0] + ".faa", "w") as out:
            for record in SeqIO.parse(open(ppath[0]), 'fasta'):
                out.write(">%s\n%s\n" %(record.description, str(record.seq)))
                all_proteins.write(">%s\n%s\n" %(record.description, str(record.seq)))
    
    all_proteins.close()

### build hmm set

In [705]:
kegg = pd.read_csv("/shared/software/kofamscan/latest/db/ko_list", sep="\t")
kegg.head()

,knum,threshold,score_type,profile_type,F-measure,nseq,nseq_used,alen,mlen,eff_nseq,re/pos,definition
0,K00001,323.77,domain,trim,0.229197,1509,1102,1939,401,19.80,0.590,alcohol dehydrogenase [EC:1.1.1.1]
1,K00002,420.60,domain,all,0.561215,1413,1367,4257,610,6.91,0.590,alcohol dehydrogenase (NADP+) [EC:1.1.1.2]
2,K00003,255.57,domain,all,0.961331,4464,3637,2225,484,6.39,0.590,homoserine dehydrogenase [EC:1.1.1.3]
3,K00004,366.27,domain,all,0.917917,939,732,809,363,3.72,0.590,"(R,R)-butanediol dehydrogenase / meso-butanedi..."
4,K00005,344.47,full,all,0.906934,1523,1093,826,387,3.06,0.590,glycerol dehydrogenase [EC:1.1.1.6]


In [706]:
kegg_hmms = {}

for group in ["cpr", "dpann"]:
    
    if group == "cpr":
        hmm_list = ["L2", "L3", "L4", "L5", "L6", "L14", "L16",
            "L15", "L18", "L22", "L24", "S3", "S8", "S10", "S17","S19"]
    else:
        hmm_list = ["L2", "L3", "L4e", "L5e", "L6", "L14", 
            "L15e", "L18", "L22", "L24e", "S3", "S8e", "S17","S19e"]
    
    with open(rootdir + "phylo/" + group + "/hmm_list.tsv", "w") as out:
        out.write("hmm_path\tthreshold\n")
        kegg_hmms[group] = {}
        for hmm in hmm_list:
            size = "small" if "S" in hmm else "large"
            subtable = kegg[kegg["definition"] == size + " subunit ribosomal protein " + hmm]
            hmm_path = glob.glob("/shared/software/kofamscan/latest/db/profiles/" + \
                subtable["knum"].iloc[0] + ".hmm")
            out.write("%s\t%s\n" %(hmm_path[0], subtable["threshold"].iloc[0]))
            kegg_hmms[group][hmm.replace("e","")] = subtable["knum"].iloc[0]

### prep ref files

In [707]:
cmdir(rootdir + "phylo/cpr/refs")
cmdir(rootdir + "phylo/dpann/refs")

In [708]:
# make dicts
hmm_dict = {"PF00410": "rpS8","PF00281":"rpL5","TIGR00060":"rpL18","TIGR01009":"rpS3",
            "TIGR01044":"rpL22","TIGR01049":"rpS10","TIGR01050":"rpS19","TIGR01067":"rpL14",
            "TIGR01071":"rpL15","TIGR01079":"rpL24","TIGR01164":"rpL16","TIGR01171":"rpL2",
            "TIGR03625":"rpL3","TIGR03635":"rpS17","TIGR03654":"rpL6","TIGR03953":"rpL4"}

In [233]:
# cpr - from Jaffe et al. 2020
scaf2bin = {}

for genome in glob.glob("/groups/banfield/users/ajaffe/cpr-dpann/nr-set-complete/dRep-workspace-95/dereplicated_genomes/*"):
    genome_name = os.path.basename(genome).split(".")[0]
    for record in SeqIO.parse(open(genome, "r"), "fasta"):
        if "UBA" in genome_name:
            scaf_name = genome_name + "_" + record.description
        else: scaf_name = record.description.split(" ")[0]
        scaf2bin[scaf_name] = genome_name
        
for file in glob.glob("/groups/banfield/users/ajaffe/cpr-dpann/nr-set-complete/trees/bac175_outgroup/[PT]*.faa"):
    
    name = os.path.basename(file).split(".")[0]
    
    if name in hmm_dict.keys():
        keggname = kegg_hmms["cpr"][hmm_dict[name].replace("rp", "")]
        with open(rootdir + "phylo/cpr/refs/" + keggname + ".CPRREF.faa", "w") as out:
            for record in SeqIO.parse(open(file), "fasta"):
                if "@" in record.description:
                    bin = record.description.split("@")[0]
                elif scaffold(record.description.split(" ")[0]) in scaf2bin.keys():
                    bin = scaf2bin[scaffold(record.description.split(" ")[0])]
                else: continue      
                out.write(">%s\n%s\n" %(bin, str(record.seq)))

In [709]:
# dpann - from Castelle et al. 2021
metadata = pd.read_csv(rootdir + "phylo/dpann/castelle_table_s1.tsv", sep="\t")
metadata["genome_name_clean"] = metadata["genome_name"].apply(lambda x: x.split(" ")[0])
# only take representative dpann and crens as outgroup
include_df = metadata[(metadata["representative genome"]=="representative") & 
    ((metadata["DPANN/non-DPANN"]=="DPANN") | (metadata["genome_name"].str.contains("Cren")))]

for file in glob.glob("/groups/banfield/projects/multienv/proteinfams/DPANN/dataset/16RP/fasta/*.fa"):

    name = os.path.basename(file).split(".")[0]
    keggname = kegg_hmms["dpann"][name]
    
    with open(rootdir + "phylo/dpann/refs/" + keggname + ".DPREF.faa", "w") as out:
        for record in SeqIO.parse(open(file), "fasta"):
            if record.description in include_df["genome_name_clean"].to_list():
                out.write(">%s\n%s\n" %(record.description, str(record.seq)))

Then head on over to rp16_pipeline.ipynb.

### annotate trees

In [751]:
color_dict = {"Woesearchaeota": "#FDCEAD", "Woesearchaeota-like I": "#5D8EB5",
             "Woesearchaeota-like II":"#F4E1B5", "Pacearchaeota": "#E1F5E1",
             "Mamarchaeota": "#E88BA0", "Parvarchaeota": "#4BD7F1",
             "Nanoarchaeota": "#7949D6", "Kleinarchaeota": "#9400D3",
             "Nanohaloarchaeota": "#378AB9", "Huberarchaeota": "#F70808",
             "Aenigmarchaeota": "#F5F5D8", "DPANN_UAP": "#C5C8CF", "Diapherotrites": "#AFD3BF",
             "Micrarchaeota": "#F4F2A8"}

In [752]:
# decorate the tree
itol = open(rootdir + "phylo/dpann/dpann.itol.txt", "w")
itol.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")

for key, row in include_df.iterrows():
    color = color_dict[row["lineage"]] if row["lineage"] in color_dict.keys() else "lightgrey"
    label = row["lineage"]
    itol.write(row["genome_name_clean"] + "\trange\t" + color + "\t" + label + "\n")
itol.close()

In [8]:
cprmd = pd.read_csv(rootdir + "phylo/cpr/jaffe_et_al_s1.tsv", sep="\t")
cprcolor = {item: random.sample(sns.color_palette("Set2").as_hex(), 1)[0] for item in cprmd["revised_tax"].unique()}

In [272]:
# decorate the tree
itol = open(rootdir + "phylo/cpr/cpr.itol.txt", "w")
itol.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")

for key, row in cprmd.iterrows():
    color = cprcolor[row["revised_tax"]] if row["revised_tax"] in cprcolor.keys() else "lightgrey"
    label = row["revised_tax"]
    itol.write(row["name"] + "\trange\t" + color + "\t" + label + "\n")
itol.close()

In [257]:
# make input spreadsheet
cpr_dpann = [os.path.basename(item).split(".")[0] + ".refined" 
    for item in glob.glob(rootdir + "phylo/*/proteins/*")]
gtresults[gtresults["user_genome"].isin(cpr_dpann)][["user_genome", "classification"]].sort_values("classification").to_csv(rootdir + "phylo/input.tsv", index=False, sep="\t")

In [6]:
# read curated output
cpr_dpann_curated = pd.read_csv(rootdir + "phylo/output.tsv", sep="\t")
cpr_dpann_curated["genome_slug"] = cpr_dpann_curated["user_genome"].apply(lambda x: x.replace(".refined", "").replace("_contigs", ""))
cpr_dpann_curated.head()

,user_genome,classification,manual,genome_slug
0,A0920SED5_metabat_64.refined,d__Archaea;p__Aenigmatarchaeota;c__Aenigmatarc...,Aenigmarchaeota,A0920SED5_metabat_64
1,A0920SED3_metabat_194.refined,d__Archaea;p__Aenigmatarchaeota;c__Aenigmatarc...,Aenigmarchaeota,A0920SED3_metabat_194
2,A0920SED5_metabat_132.refined,d__Archaea;p__Aenigmatarchaeota;c__Aenigmatarc...,Aenigmarchaeota,A0920SED5_metabat_132
3,A0920SED5_metabat_449.refined,d__Archaea;p__Aenigmatarchaeota;c__Aenigmatarc...,Aenigmarchaeota,A0920SED5_metabat_449
4,A0920SED5_metabat_1.refined,d__Archaea;p__Aenigmatarchaeota;c__Aenigmatarc...,Aenigmarchaeota,A0920SED5_metabat_1


### re-annotate for figure

In [6]:
# read in new color palette
new_palette = {row["lineage"]: row["color"] for key, row in pd.read_csv(rootdir + "tables/cpr_colors.tsv", sep="\t").iterrows()}
new_palette

{'Parcubacteria_3': '#8dd3c7',
 'Parcubacteria_4': '#ffffb3',
 'Peregrinibacteria': '#bebada',
 'other': 'lightgrey',
 'Absconditabacteria': '#80b1d3',
 'Saccharibacteria': '#fdb462',
 'Parcubacteria_1': '#b3de69',
 'Berkelbacteria': '#fccde5',
 'Parcubacteria_2': '#d9d9d9',
 'Moranbacteria': '#bc80bd',
 'Microgenomates': '#ccebc5'}

In [31]:
# read in tree order
tree_order = pd.read_csv("/groups/banfield/users/ajaffe/cpr-dpann/nr-set-complete/trees/tree_order.txt", header=None).fillna("None")
tree_order.columns = ["lineage", "group"]
tree_order.head()

,lineage,group
0,Dojkabacteria,None
1,WWE3,Katanobacteria (WWE3)
2,Woykebacteria,Microgenomates
3,Curtissbacteria,Microgenomates
4,Daviesbacteria,Microgenomates


In [34]:
cprmd_sub = cprmd[["name", "revised_tax"]]
curated_sub = cpr_dpann_curated[cpr_dpann_curated["classification"].str.contains("Patescibacteria")][["genome_slug", "manual"]]
curated_sub.columns = ["name", "revised_tax"]
combined = pd.concat([cprmd_sub, curated_sub])
combined = combined.merge(tree_order, how="left", left_on="revised_tax", right_on="lineage").fillna("None")
combined["revised_group"] = combined.apply(lambda x: x["group"] if x["group"]!="None" else x["revised_tax"], axis=1)
combined.head()

,name,revised_tax,lineage,group,revised_group
0,BJP_IG2102_PER_44_74_final,Peregrinibacteria,Peregrinibacteria,Peregrinibacteria,Peregrinibacteria
1,BJP_08E140C01_Falkowbacteria_33_19_curated,Falkowbacteria,Falkowbacteria,Parcubacteria_1,Parcubacteria_1
2,BJP_IG2103_TM7_39_1400_curated,Saccharibacteria,Saccharibacteria,Saccharibacteria,Saccharibacteria
3,UBA661,Kaiserbacteria,Kaiserbacteria,Parcubacteria_4,Parcubacteria_4
4,UBA669,Saccharibacteria,Saccharibacteria,Saccharibacteria,Saccharibacteria


In [57]:
# temporary fixes
adds = {"Parcubacteria_1": ("Candidatus_Jacksonbacteria_bacterium_RIFCSPLOWO2_01_FULL_44_13", "CG_4_10_14_0_8_um_filter_cor_Magasanikbacteria_OD1_42_12_curated"),
       "Parcubacteria_2": ("Candidatus_Spechtbacteria_bacterium_RIFCSPHIGHO2_02_FULL_43_15b", "CG08_land_8_20_14_0_20_Nealsonbacteria_OD1_38_20_curated"),
       "Parcubacteria_3": ("A0920SED4_metabat_56", "CG10_big_fil_rev_8_21_14_0_10_Harrisonbacteria_OD1_45_28_curated"),
       "Parcubacteria_4": ("Parcubacteria_group_bacterium_RIFCSPHIGHO2_01_FULL_45_26", "A0920WC90_metabat_50")}

In [60]:
# decorate the tree
itol = open(rootdir + "figures/cpr.itol.final.txt", "w")
itol.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")

for key, row in combined.iterrows():
    if "Parcubacteria" not in row["revised_group"]:
        color = new_palette[row["revised_group"]] if row["revised_group"] in new_palette else "white"
        label = row["revised_group"]
        if "Lac" in row["name"]:
                newname = row["name"] + "_contigs"
        else: newname = row["name"]
        itol.write(newname + "\trange\t" + color + "\t" + label + "\n")
    
for key in adds.keys():
    color = new_palette[key]
    itol.write(adds[key][0] + "|" + adds[key][1] + "\trange\t" + color + "\t" + key + "\n")
itol.close()

In [56]:
# make points to single out LP genomes
symbols = open(rootdir + "figures/cpr.itol.symbols.txt", "w")
symbols.write("DATASET_BINARY\nSEPARATOR COMMA\nDATASET_LABEL,LP\nCOLOR, \
    #ffff00\nFIELD_SHAPES,2\nFIELD_LABELS,LP\nDATA\n")

for key, row in combined.iterrows():
    if ("Lac" in row["name"]) or ("A0" in row["name"]):
        if "Lac" in row["name"]:
            newname = row["name"] + "_contigs"
        else: newname = row["name"]
        symbols.write(newname + ",1\n")
symbols.close()

# finalize

In [309]:
cmdir(rootdir + "ggkbase/dropped_scaf_rebin")

### batch rebin dropped scafs

In [712]:
samplescafs = {}
count=0

for bin in glob.glob(rootdir + "genomes/prelim_genomes/*"):
        
    # get sample identifier
    if "LacPavin" in os.path.basename(bin):
        slug = "_".join(os.path.basename(bin).split("_")[0:3])
    else:
        m = re.search("A([0-9]*)([A-Z0-9]*)", os.path.basename(bin))
        slug = "LacPavin_%s_%s" %(m.group(1), m.group(2))

    # find new genome
    new_genome = glob.glob(rootdir + "genomes/final_genomes/" + \
        os.path.basename(bin).split(".fa")[0].replace(".", "_") + ".*")

    if len(new_genome) == 1:

        old_scafs = [record.description.split(" ")[0] for record in SeqIO.parse(open(bin), "fasta")]
        new_scafs = [record.description.split(" ")[0] for record in SeqIO.parse(open(new_genome[0]), "fasta")]
        scafdif = set(old_scafs).difference(new_scafs)

        if slug not in samplescafs.keys():
            samplescafs[slug] = list(scafdif)
        else: samplescafs[slug] += list(scafdif)

In [316]:
for sample in samplescafs.keys():
    with open(rootdir + "ggkbase/dropped_scaf_rebin/" + sample + ".tsv", "w") as out:
        for scaf in samplescafs[sample]:
            out.write("%s\n" %(scaf))

### merge and rename organisms

In [561]:
import string
cmdir(rootdir + "ggkbase/refined_org_tables")

In [730]:
orgtables = pd.concat([pd.read_csv(item, sep="\t") for item in glob.glob(rootdir + "ggkbase/refined_org_tables/*")])
orgsub = orgtables[["name", "binning project", "taxonomy", "bin length", "GC%", "coverage", "# contigs"]]
orgsub["genome_slug"] = orgsub["name"].apply(lambda x: x.replace("_sub", "") if x.startswith("A") else x)
orgmerge = orgsub.merge(cpr_dpann_curated[["genome_slug", "manual"]], how="left", on="genome_slug").fillna("None")
orgmerge = orgmerge.merge(gtresults[["genome_slug", "classification"]], how="left", on="genome_slug").fillna("None")
orgmerge["cpr_dpann"] = orgmerge["manual"].apply(lambda x: True if x !="None" else False)
orgmerge = orgmerge[orgmerge["classification"] != "None"]

In [731]:
# check len - lots of naming issues
len(orgmerge)

738

In [732]:
existing = []

def construct_name(row, existing):
    
    if row["cpr_dpann"] == True:
        token = row["manual"]
    
    elif (row["taxonomy"] != "None") & ("metagenome" not in row["taxonomy"]):
        
        # take first non sp. name
        token = ""
        for item in row["taxonomy"].split(","):
            clean = item.strip().replace("Candidatus ", "")
            if ("_" not in clean) & (len(clean.split(" "))==1):
                token=clean
                break 
    else:
        token = row["classification"].split(";")[0].replace("d__", "")
    
    proposed = "_".join([row["binning project"], token, 
        str(int(round(row["GC%"],0))), str(round(row["coverage"]))])
    
    # amend duplicates
    count = existing.count(proposed)
    if count > 0:
        proposed += list(string.ascii_lowercase)[count]
    else:
        existing.append(proposed)
    
    return proposed

orgmerge["newname"] = orgmerge.apply(lambda x: construct_name(x, existing), axis=1)

In [733]:
# check names are unique
orgmerge["newname"].value_counts()[0:5]

LacPavin_0818_WC50_Burkholderiales_53_5    1
LacPavin_0920_SED5_Bacteria_66_35          1
LacPavin_0920_SED3_Bacteria_65_12          1
LacPavin_0920_WC12_Cyanobium_62_9          1
LacPavin_0818_WC50_Actinobacteria_57_5     1
Name: newname, dtype: int64

In [734]:
# check number of fields is consistent
set(len(item.split("_")) for item in orgmerge["newname"].unique())

{6}

### export

In [631]:
cmdir(rootdir + "genomes/renamed_genomes")

In [738]:
# biotite
for key, row in orgmerge.iterrows():
    
    # find fasta
    path = glob.glob(rootdir + "genomes/final_genomes/" + row["genome_slug"] + "[._]*")
    # write fasta
    with open(rootdir + "genomes/renamed_genomes/" + row["newname"] + ".fna", "w") as out:
        for record in SeqIO.parse(open(path[0]), "fasta"):
            out.write(">%s\n%s\n" %(record.description.split(" ")[0], str(record.seq)))

In [739]:
# old->new quick mapping
orgmerge[["name", "newname"]].to_csv(rootdir + "genomes/genome_name_mapping.tsv", sep="\t", index=False, header=None)

In [740]:
# ggkbase rename
with open(rootdir + "ggkbase/genome_rename.tsv", "w") as out:
    for key, row in orgmerge.iterrows():
        out.write("%s\t%s\t%s\n" %(row["name"], row["binning project"], row["newname"]))

In [741]:
# ggkbase AP
orgmerge["newname"].to_csv(rootdir + "ggkbase/analysis_project.txt", index=False, header=None)

### supp table 1 base

In [742]:
cmdir(rootdir + "tables")

In [786]:
table_s1 = orgmerge.rename(columns={"newname": "genome_name", "binning project": "sample",
                         "taxonomy": "ggkbase_taxonomy", "bin length": "bin_length", 
                         "GC%": "gc_perc", "# contigs": "num_scaffolds", "manual": "cpr_dpann_lineage",
                         "classification": "gtdbtk_taxonomy"})
curation_data = pd.read_csv(rootdir + "tables/curation_data.tsv", sep="\t")
# overwrite old genome slug to make compatible :/
curation_data["genome_slug"] = curation_data["bin"].apply(lambda x: x.replace("_contigs", ""))
curation_data = curation_data.rename(columns={"new_completeness": "checkm_completeness", 
    "new_contamination": "checkm_redundancy", "css_refined": "gunc_phylum_css"})
table_s1 = table_s1.merge(curation_data[["genome_slug", "checkm_completeness", "checkm_redundancy", 
    "gunc_phylum_css"]], how="left", on="genome_slug").fillna("None")
# TODO: scaffold n50?
# reorganize
table_s1 = table_s1[["genome_name", "sample", "cpr_dpann", "cpr_dpann_lineage", "ggkbase_taxonomy", 
                    "gtdbtk_taxonomy", "bin_length", "num_scaffolds", "gc_perc", "checkm_completeness",
                    "checkm_redundancy", "gunc_phylum_css"]]
# sort on gtdbtk d + p?
table_s1["sort"] = table_s1["gtdbtk_taxonomy"].apply(lambda x: ";".join(x.split(";")[0:2]))
table_s1 = table_s1.sort_values(["sample", "sort"]).drop(["sort"], axis=1)
table_s1.head()

,genome_name,sample,cpr_dpann,cpr_dpann_lineage,ggkbase_taxonomy,gtdbtk_taxonomy,bin_length,num_scaffolds,gc_perc,checkm_completeness,checkm_redundancy,gunc_phylum_css
147,LacPavin_0419_WC53_Bacteria_61_8,LacPavin_0419_WC53,False,None,Bacteria,d__Bacteria;p__Acidobacteriota;c__Acidobacteri...,7913254,574,61.07,96.96,6.52,0.15
126,LacPavin_0419_WC53_Rhodococcus_63_12,LacPavin_0419_WC53,False,None,"Rhodococcus, Corynebacteriales, Actinobacteria...",d__Bacteria;p__Actinobacteriota;c__Actinomycet...,6080610,483,62.64,98.70,1.64,0.00
130,LacPavin_0419_WC53_Actinobacteria_62_15,LacPavin_0419_WC53,False,None,"Actinobacteria, Actinobacteria, Bacteria",d__Bacteria;p__Actinobacteriota;c__Acidimicrob...,2702562,346,61.58,85.04,1.80,0.21
140,LacPavin_0419_WC53_Rhodococcus_65_21,LacPavin_0419_WC53,False,None,"Rhodococcus sp. PML026, Rhodococcus, Corynebac...",d__Bacteria;p__Actinobacteriota;c__Actinomycet...,4807012,582,64.87,97.25,3.54,0.00
145,LacPavin_0419_WC53_Actinobacteria_58_7,LacPavin_0419_WC53,False,None,"actinobacterium acAMD-2, Actinobacteria, Actin...",d__Bacteria;p__Actinobacteriota;c__Actinomycet...,1496122,252,57.52,80.46,1.62,0.32


Export to later add onto in a separate NCBI upload notebook.

In [787]:
table_s1.to_csv(rootdir + "tables/jaffe_table_s1.tsv", sep="\t", index=False)